# Intro to TensorFlow

(Adapted based on content from TensorFlow documentation.)

TensorFlow is a open source library, built by Google's Brain Team, used in building neural networks and deep learning frameworks. The core of TensorFlow is the computation graph (compute graph) that allows data to flow as inputs, through nodes (operation), which are connected by edges (multidimensional data arrays called tensors) to generate some sort of output, either a classification or numerical output.

The low-level API is called __TensorCore__ ideal for someone who needs high control over their models, such as machine learning researchers. The high-level API is in the `tf.contrib.learn` library which is a library constantly under development, thus, functions and methods available today might not be available down the road as the library continues to evolve.

### Tensors

__Tensors__ are units of data arranged into an array. A tensor's __rank__ is the number of dimensions of a tensor.

In [74]:
[1,2,3] # rank 1 with shape [3]
[[1,2,3], [1,2,3]] # rank 2 with shape [2,3]
[[[1,2,3]], [[1,2,3]]] # rank 3 with shape [2,1,3]

[[[1, 2, 3]], [[1, 2, 3]]]

Before we continue, let's import tensorflow into our environment which will give us access to all the functionality of tensorflow.

In [4]:
import tensorflow as tf

### Compute Graph

A __computational graph__ is the core of the tf library. It is a series of operations arranged into graph nodes that the graph can then run. This is a two step process: first you build the graph, outlining the operations and inputs, then you can run the graph to get the outputs.

Below are 2 nodes built into the graph.

In [75]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_6:0", shape=(), dtype=float32) Tensor("Const_7:0", shape=(), dtype=float32)


Both nodes are constants, that is, they do not change. Additionally, we can see that when we print the nodes their values don't show. This is because to get the values we need to run the nodes within a __session__. A session encapsulates the control and state of the tf runtime. First we will create a session and then envoke the run method one it.

### Sessions

sess = tf.Session()
print(sess.run([node1, node2]))

Now we have the values of both constants. We can even add these constants together in a third node.

In [76]:
node3 = tf.add(node1, node2)
print("node3", node3)
print("sess.run(node3):", sess.run(node3))

node3 Tensor("Add_2:0", shape=(), dtype=float32)
sess.run(node3): 7.0


### Placeholders and operations

A graph can be parameterized to accept external inputs which are placeholders, a promise to provide a value later. We can produce a function, similiar to `tf.add` which will add the nodes together.

In [77]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

Now we can feed the `sess.run` a function `adder_node` and a dictionary of values to compute.

In [78]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


We can continue to make the graph complex by adding more operations.

In [79]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


### Models 

__Variables__ will allow us to add trainable paramters to the graph. Each variable will be defined by a `dtype` and intial value.

In [80]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Note: Constants are intialized and immutable after calling `tf.constant` but variables are not intialized using `tf.Variable`. To intialize the variables, we need to call `tf.global_variables_initializer()` which intializes all variables.

In [81]:
init = tf.global_variables_initializer()
sess.run(init)

Once we call `sess.run()` on the intializer, the variables are initialized. 

Since we create `x` as a placeholder, we can pass a dictionary of values to `sess.run` to see how each evaluate.

In [82]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


### Model Evaluation

Now that the model is created, we need to evaluate the effectiveness of the model. To do this we will create a `y` variable and a __loss__ function to evaluate how good our model is. The loss function we will use to evaluate our model is the standard linear regression evaluator,  mean squared error (MSE).

In [83]:
y = tf.placeholder(tf.float32)
squared_errors = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_errors)
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

23.66


Above, we fed the session the `loss` function which requires `x` values to evaluate `linear_model` and `y` values to compute `square_errors`. Feeding `sess.run` the loss function and a dictionary of both `x` and `y` values gives us a value of 23.66.

The `tf.reduce_sum` method collapses the 1d of shape 4 to a 1d tensor of shape 1 by summing the values across the tensor.

We can look below at a more basic implementation of the model with the same values.

In [84]:
val = [0., 0.30000001, 0.60000002, 0.90000004]
new_y = [0., -1., -2., -3.]
new = 0
for num in range(4):
    n = val[num] - new_y[num]
    n = n**2
    new += n
print(new)

23.660000442000005


### Model Tuning

As we can see, we have a MSE of 23.66. We could improve this model by tuning the parameters. In this basic example, we can get a perfect model by just reassign the values of `W` and `b` to -1 and 1. We will use `tf.assign` to make the adjustments.

In [86]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


Now we can see that the model is perfect, our MSE is 0.0. However, this is not how we would likely do this. In machine learning and predictive modeling, we expect the model to learn to new parameters on their own to improve the MSE and reduce the loss.

### Using optimizers

Here will will use an optimizer from the `tf.train` API. The optimizer we will use will be __gradient descent__ which searches for the local minimum, thus tuning the variable to find it's most optimal state to reduce loss. 

First we will initilize the Gradient Descent Optimizer with a learning rate of `0.01`. The learning rate determines the speed at which we move toward the optimial paramters. Since we are model is fairly simple, we can use a small learning rate with 1000 iterations to increase our chances of converging to the optimal solution.

In [88]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [92]:
sess.run(init) # resets values to incorrect defaults
for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


The output are the final model paramters for the simple linear model we have built using TensorFlow. As we can see, they are awfully close to our manual tuning when we assigned -1 and 1 as the values of `W` and `b`.

### Closing

In all, we can see similiarities between the model building process of `TensorFlow` and the defacto python data science library `scikit-learn`, in that we are defining our model structure (given that our example was a parametric model), provide inputs and outcomes, and optimize the parameters to find the best fit using an optmizer, in our case gradient descent.

At it's most basic level, the most noticeable difference is that we define the model structure in TensorFlow and then run the model using a the `run` method of a TensorFlow session. The whole process is contained in a session. 

Note, this whole process is a very basic intro to TensorFlow. The TensorFlow library is already robust and ready for use in deep learning production for projects such as image recognition. Additionally, the library is constantly growing and improving given the support in the community. A.